## Command

An object which represents an instruction to perform a particular
action. Contains all the information necessary for the action
to be taken.

### Command

In [5]:
from enum import Enum
from abc import ABC, abstractmethod

class BankAccount:
    OVERDRAFT_LIMIT = - 500
    
    def __init__(self, balance=0):
        self.balance = balance
    
    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance={self.balance}')
    
    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance={self.balance}')
            return True
        return False
        
        
    def __str__(self):
        return f'Balance={self.balance}'
    

class Command(ABC):
    @abstractmethod
    def invoke(self):
        ...
    @abstractmethod
    def undo(self):
        ...
    
        
class BankAccountCommand(Command):
    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1
    
    def __init__(self, account, action, amount):
        self.account = account
        self.action = action
        self.amount = amount
        self.success = None
        
    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)
            
    def undo(self):
        if not self.success:
            return 
        
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)
            
            
ba = BankAccount() # 0
cmd = BankAccountCommand(
    ba, BankAccountCommand.Action.DEPOSIT, 100
)

cmd.invoke()
print(f'After $100 deposit: {ba}\n')

cmd.undo()
print(f'$100 deposit undone: {ba}\n')

illegal_cmd = BankAccountCommand(
    ba, BankAccountCommand.Action.WITHDRAW, 1000
)
illegal_cmd.invoke()
print(f'After impossible withdrawal: {ba}\n')

illegal_cmd.undo()
print(f'After undo: {ba}\n')


Deposited 100, balance=100
After $100 deposit: Balance=100

Withdrew 100, balance=0
$100 deposit undone: Balance=0

After impossible withdrawal: Balance=0

After undo: Balance=0



### Composite Command

In [12]:
from enum import Enum
from abc import ABC, abstractmethod
from unittest import TestCase, TextTestRunner, defaultTestLoader


class BankAccount:
    OVERDRAFT_LIMIT = - 500
    
    def __init__(self, balance=0):
        self.balance = balance
    
    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance={self.balance}')
    
    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance={self.balance}')
            return True
        return False
        
        
    def __str__(self):
        return f'Balance={self.balance}'
    

class Command(ABC):
    def __init__(self):
        self.success = False    
    
    @abstractmethod
    def invoke(self): ...
        
    @abstractmethod
    def undo(self): ...
    
        
class BankAccountCommand(Command):
    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1
    
    def __init__(self, account, action, amount):
        super().__init__()
        self.account = account
        self.action = action
        self.amount = amount
        
    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)
            
    def undo(self):
        if not self.success:
            return 
        
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)
            
            
class CompositeBankAccountCommand(Command, list):
    def __init__(self, items=[]):
        super().__init__()
        for i in items:
            self.append(i)
            
    def invoke(self):
        for x in self:
            x.invoke()
            
    def undo(self):
        for x in reversed(self):
            x.undo()


class MoneyTransferCommand(CompositeBankAccountCommand):
    def __init__(self, from_acct, to_acct, amount):
        super().__init__([
            BankAccountCommand(
                from_acct,
                BankAccountCommand.Action.WITHDRAW,
                amount,
            ),
            BankAccountCommand(
                to_acct,
                BankAccountCommand.Action.DEPOSIT,
                amount,
            )
        ])
    def invoke(self):
        ok = True
        for cmd in self:
            if ok:
                cmd.invoke()
                ok = cmd.success
            else:
                cmd.success = False
        self.success = ok

            

class TestSuite(TestCase):
#     def test_transfer_fail(self):
#         ba1 = BankAccount(100)
#         ba2 = BankAccount()
        
#         amount = 1000
#         wc = BankAccountCommand(
#             ba1, BankAccountCommand.Action.WITHDRAW, amount
#         )
#         dc = BankAccountCommand(
#             ba2, BankAccountCommand.Action.DEPOSIT, amount
#         )
        
#         transfer = CompositeBankAccountCommand([wc, dc])
#         transfer.invoke()
#         print(f'ba1: {ba1}\nba2: {ba2}\n\n')
#         transfer.undo()
#         print(f'ba1: {ba1}\nba2: {ba2}')
    def test_better_transfer(self):
        ba1 = BankAccount(100)
        ba2 = BankAccount()
        
        amount = 1000
        transfer = MoneyTransferCommand(ba1, ba2, amount)
        transfer.invoke()
        print(f'ba1: {ba1}\nba2: {ba2}\n\n') 
        transfer.undo()
        print(f'ba1: {ba1}\nba2: {ba2}')        
        print(transfer.success)
        
        
        
TextTestRunner().run(
    defaultTestLoader.loadTestsFromTestCase(TestSuite)
)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


ba1: Balance=100
ba2: Balance=0


ba1: Balance=100
ba2: Balance=0
False


<unittest.runner.TextTestResult run=1 errors=0 failures=0>